# 🍺 Synthetische Datengenerierung für ein innovatives Bier (Genussfokus, vollständiger Fragebogen)
Dieses Notebook nutzt ein Conditional GAN (CGAN), um realistische, genussorientierte Bierkonsum-Personas basierend auf dem vollständigen Fragebogen zu generieren. Alle Fragen werden automatisch numerisch encodiert. Die Mapping-Tabellen erlauben eine Rücktransformation nach der Generierung.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers
import json
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('data/Köhler_Iselborn_Rohdatensatz_Zenodo.csv')

# Nur abgeschlossene Antworten
df = df[df['Status'] == 'Completed'].copy()

# Nicht benötigte Spalten entfernen
df.drop(columns=[
    'Fall', 'Status', 'Bearbeitungszeit(Sekunden)',
    'ErklärungDatenverarbeitung', 'Q0', 'Q3'
], inplace=True)

# -99 als NaN behandeln
df.replace(-99, np.nan, inplace=True)

# Alle Spalten in nullable Integer (Int64) umwandeln
df = df.astype('Int64')

# Optional speichern
df.to_csv("clean_data.csv", index=False)
df

,Q1,Q2A,Q2B,Q2C,Q2D,Q2E,Q2F,Q2G,Q4A,Q4B,...,Q12I,Q13,Q14,Q15,Q16,Q17A,Q17B,Q17C,Q17D,Q17E
0,5,2,2,1,1,1,2,1,2,2,...,3,2,6,1,3,0,0,1,0,0
1,3,2,2,1,1,2,2,1,2,2,...,2,1,6,1,3,0,0,1,0,0
2,3,3,3,2,2,1,2,2,3,2,...,4,2,8,2,1,0,0,0,1,0
4,3,2,2,1,2,2,2,1,2,2,...,2,1,5,1,3,0,0,1,0,0
7,2,2,3,1,2,3,3,3,3,3,...,5,1,5,1,3,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,3,3,3,1,1,3,3,3,2,3,...,3,1,8,1,9,0,0,1,0,0
532,2,3,3,2,1,3,3,2,3,2,...,5,2,7,1,3,0,0,1,0,1
533,3,2,3,1,1,3,3,3,1,3,...,1,2,12,2,1,1,0,0,0,0
534,5,1,3,1,1,3,1,1,2,2,...,2,2,6,1,3,1,0,0,0,0


In [5]:
# Genussrelevante Fragen
positive_focus = ['Q2C','Q2D','Q2G','Q8A','Q8B','Q6C']
negative_focus = ['Q8C','Q6F']

def genuss_score(row):
    score = 0
    for col in positive_focus:
        score += row.get(col, 0) if pd.notna(row.get(col)) else 0
    for col in negative_focus:
        score -= row.get(col, 0) if pd.notna(row.get(col)) else 0
    return score

df['genuss_score'] = df.apply(genuss_score, axis=1)
df

,Q1,Q2A,Q2B,Q2C,Q2D,Q2E,Q2F,Q2G,Q4A,Q4B,...,Q13,Q14,Q15,Q16,Q17A,Q17B,Q17C,Q17D,Q17E,genuss_score
0,5,2,2,1,1,1,2,1,2,2,...,2,6,1,3,0,0,1,0,0,11
1,3,2,2,1,1,2,2,1,2,2,...,1,6,1,3,0,0,1,0,0,9
2,3,3,3,2,2,1,2,2,3,2,...,2,8,2,1,0,0,0,1,0,16
4,3,2,2,1,2,2,2,1,2,2,...,1,5,1,3,0,0,1,0,0,8
7,2,2,3,1,2,3,3,3,3,3,...,1,5,1,3,0,0,1,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,3,3,3,1,1,3,3,3,2,3,...,1,8,1,9,0,0,1,0,0,8
532,2,3,3,2,1,3,3,2,3,2,...,2,7,1,3,0,0,1,0,1,10
533,3,2,3,1,1,3,3,3,1,3,...,2,12,2,1,1,0,0,0,0,12
534,5,1,3,1,1,3,1,1,2,2,...,2,6,1,3,1,0,0,0,0,5


In [6]:
# Zielgruppe = oberes Quartil des Genuss-Scores
threshold = df['genuss_score'].quantile(0.75)
df_target = df[df['genuss_score'] >= threshold].copy()
df_target

,Q1,Q2A,Q2B,Q2C,Q2D,Q2E,Q2F,Q2G,Q4A,Q4B,...,Q13,Q14,Q15,Q16,Q17A,Q17B,Q17C,Q17D,Q17E,genuss_score
2,3,3,3,2,2,1,2,2,3,2,...,2,8,2,1,0,0,0,1,0,16
8,3,2,3,1,1,2,3,2,3,1,...,1,8,1,3,0,0,0,0,1,13
15,2,3,3,2,1,3,3,3,3,2,...,1,7,1,3,0,0,1,0,0,14
16,2,3,3,2,1,3,3,3,3,2,...,1,5,1,3,0,0,1,0,0,13
30,1,3,3,3,3,3,3,3,3,3,...,1,10,1,3,1,0,0,0,0,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,3,2,3,1,1,1,2,3,3,1,...,1,5,1,3,0,0,1,0,0,13
529,2,3,3,2,2,3,3,3,3,3,...,1,15,1,1,0,0,0,1,0,12
530,2,3,3,2,2,3,3,3,2,3,...,1,11,1,1,0,0,1,0,0,13
533,3,2,3,1,1,3,3,3,1,3,...,2,12,2,1,1,0,0,0,0,12
